In [ ]:
import freetype
import numpy as np
from scipy.special import binom
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches

In [ ]:
%load_ext autoreload
%autoreload 2
import text

In [ ]:
face = freetype.Face("Vera.ttf")
face.set_char_size(48 * 64)
face.load_char("g")
slot = face.glyph

outline = slot.outline
points = np.array(outline.points, dtype=[("x", float), ("y", float)])
x, y = points["x"], points["y"]

start, end = 0, 0
VERTS, CODES = [], []
# Iterate over each contour
for i in range(len(outline.contours)):
    end = outline.contours[i]
    points = outline.points[start : end + 1]
    points.append(points[0])
    tags = outline.tags[start : end + 1]
    tags.append(tags[0])

    segments = [
        [
            points[0],
        ],
    ]
    for j in range(1, len(points)):
        segments[-1].append(points[j])
        if tags[j] & (1 << 0) and j < (len(points) - 1):
            segments.append(
                [
                    points[j],
                ]
            )
    verts = [
        points[0],
    ]
    codes = [
        Path.MOVETO,
    ]
    for segment in segments:
        if len(segment) == 2:
            verts.extend(segment[1:])
            codes.extend([Path.LINETO])
        elif len(segment) == 3:
            verts.extend(segment[1:])
            codes.extend([Path.CURVE3, Path.CURVE3])
        else:
            print(len(segment))
            verts.append(segment[1])
            codes.append(Path.CURVE3)
            for i in range(1, len(segment) - 2):
                A, B = segment[i], segment[i + 1]
                C = ((A[0] + B[0]) / 2.0, (A[1] + B[1]) / 2.0)
                verts.extend([C, B])
                codes.extend([Path.CURVE3, Path.CURVE3])
            verts.append(segment[-1])
            codes.append(Path.CURVE3)
    VERTS.extend(verts)
    CODES.extend(codes)
    start = end + 1

In [ ]:
segments

In [ ]:
figure = plt.figure(figsize=(8, 10))
axis = figure.add_subplot(111)
# axis.scatter(points['x'], points['y'], alpha=.25)
axis.scatter(x, y)

# Draw glyph lines
path = Path(VERTS, CODES)
glyph = patches.PathPatch(path, facecolor=".75", lw=1)

# Draw "control" lines
for i, code in enumerate(CODES):
    if code == Path.CURVE3:
        CODES[i] = Path.LINETO
path = Path(VERTS, CODES)
patch = patches.PathPatch(path, ec=".5", fill=False, ls="dashed", lw=1)

axis.add_patch(patch)
axis.add_patch(glyph)

axis.set_xlim(x.min() - 100, x.max() + 100)
plt.xticks([])
axis.set_ylim(y.min() - 100, y.max() + 100)
plt.yticks([])
plt.show()

In [ ]:
VERTS

In [ ]:
CODES

In [ ]:
slot.metrics.vertAdvance

In [ ]:
DEFAULT_FACE = freetype.Face("Vera.ttf")

In [ ]:
DEFAULT_FACE.max_advance_height

In [ ]:
DEFAULT_FACE.units_per_EM

In [ ]:
DEFAULT_FACE.height

In [ ]:
def Text2(
    text,
    size,
    position=(0, 0),
    horizontal=True,
    angle=0,
    x_reflection=False,
    layer=0,
    datatype=0,
    face=DEFAULT_FACE,
):
    offset = np.zeros(2)
    # face.set_char_size( 48*64 )
    face.set_char_size(1)
    for char in text:
        if char == "\n":
            offset[0] = 0
            offset[1] -= slot.metrics.vertAdvance
            continue
        elif char == " ":
            offset[0] += slot.metrics.horiAdvance
            continue
        print('loading "{}"'.format(char))
        face.load_char(char, freetype.FT_LOAD_NO_SCALE)
        # face.load_char(char)
        slot = face.glyph
        outline = slot.outline
        points = np.array(outline.points).T + offset[:, np.newaxis]
        plt.scatter(*points)
        print(points.shape)
        patch = patches.Polygon(points.T, color="g", closed=True)
        plt.gca().add_patch(patch)
        offset[0] += slot.metrics.horiAdvance


# plt.figure(figsize=(20,20))
# Text2("this is a\ntest of text rendering", 10)
Text2("AiaodgLljk", 10)

In [ ]:
face.load_char("i")

In [ ]:
slot.outline.tags

In [ ]:
slot.outline.tags

In [ ]:
face = freetype.Face("Vera.ttf")
face.set_char_size(48 * 64)
face.load_char("s")
slot = face.glyph
outline = slot.outline
contours = np.array(outline.contours)
for start, end in zip(np.concatenate(((0,), contours[:-1] + 1)), contours):
    print(start, end)
    points = outline.points[start : end + 1]
    # points.append(points[0])
    tags = outline.tags[start : end + 1]
    # tags.append(tags[0])
    print(points)
    print("tags", tags)
    print("---")
    # segments = [ [points[0],], ]
    # SEE: https://www.freetype.org/freetype2/docs/glyphs/glyphs-6.html
    segments = []
    for j in range(len(points)):  # range(1, len(points)-1):
        if (
            tags[j] & freetype.FT_CURVE_TAG_CONIC
            and j > 1
            and tags[j - 1] & freetype.FT_CURVE_TAG_CONIC
        ):
            # create virtual on point at midpoint of points[j-1] and points[j]
            midpoint = (points[j - 1] + points[j]) / 2
            segments[-1].append(midpoint)
            segments.append([midpoint, points[j]])
        else:
            if j == 0 and tags[0] & freetype.FT_CURVE_TAG_CONIC:
                # first point is off point, so segment should start at last point in contour
                if tags[-1] & freetype.FT_CURVE_TAG_CONIC:
                    # last point is also off point, so segment should start and end with
                    # midpoint of first and last points
                    midpoint = (points[0] + points[-1]) / 2
                    segments.append([midpoint, points[0]])
                else:
                    segments.append([points[-1], points[0]])
            if j != 0:
                segments[-1].append(points[j])
            if tags[j] & freetype.FT_CURVE_TAG_ON:
                segments.append([points[j]])
    if tags[0] & freetype.FT_CURVE_TAG_CONIC and tags[-1] & freetype.FT_CURVE_TAG_CONIC:
        # first and last points are off points, so last on point is midpoint of first and last points
        midpoint = (points[0] + points[-1]) / 2
        segments[-1].append[midpoint]
    else:
        # close last segment with first point
        segments[-1].append(points[0])
    # uuu
    print("segments", segments)
    for seg in segments:
        if len(seg) == 2:
            plt.plot(*np.array(seg).T)
        elif len(seg) == 3 or len(seg) == 4:
            plt.plot(*text.bezier(np.array(seg), np.linspace(0, 1, 100)).T)
        else:
            print("WHOOPS", len(seg))

In [ ]:
face = freetype.Face("Vera.ttf")
face.set_char_size(48 * 64)
face.load_char("s")
slot = face.glyph
outline = slot.outline
contours = np.array(outline.contours)
polygons = []
for start, end in zip(np.concatenate(((0,), contours[:-1] + 1)), contours):
    points = outline.points[start : end + 1]
    tags = outline.tags[start : end + 1]
    # SEE: https://www.freetype.org/freetype2/docs/glyphs/glyphs-6.html
    segments = []
    for j in range(len(points)):  # range(1, len(points)-1):
        if (
            tags[j] & freetype.FT_CURVE_TAG_CONIC
            and j > 1
            and tags[j - 1] & freetype.FT_CURVE_TAG_CONIC
        ):
            # create virtual on point at midpoint of points[j-1] and points[j]
            midpoint = (points[j - 1] + points[j]) / 2
            segments[-1].append(midpoint)
            segments.append([midpoint, points[j]])
        else:
            if j == 0 and tags[0] & freetype.FT_CURVE_TAG_CONIC:
                # first point is off point, so segment should start at last point in contour
                if tags[-1] & freetype.FT_CURVE_TAG_CONIC:
                    # last point is also off point, so segment should start and end with
                    # midpoint of first and last points
                    midpoint = (points[0] + points[-1]) / 2
                    segments.append([midpoint, points[0]])
                else:
                    segments.append([points[-1], points[0]])
            if j != 0:
                segments[-1].append(points[j])
            if tags[j] & freetype.FT_CURVE_TAG_ON:
                segments.append([points[j]])
    if tags[0] & freetype.FT_CURVE_TAG_CONIC and tags[-1] & freetype.FT_CURVE_TAG_CONIC:
        # first and last points are off points, so last on point is midpoint of first and last points
        midpoint = (points[0] + points[-1]) / 2
        segments[-1].append[midpoint]
    else:
        # close last segment with first point
        segments[-1].append(points[0])
    polygon_segments = []
    for segment in segments:
        if len(segment) == 2:
            # plt.plot(*np.array(segment).T)
            polygon_segments.append((segment[0],))
        elif len(segment) == 3 or len(segment) == 4:
            # plt.plot(*text.bezier(np.array(segment), np.linspace(0, 1, 100)).T)
            polygon_segments.append(
                text.bezier(np.array(segment), np.linspace(0, 1, 100))[:-1]
            )
        else:
            raise ValueError(
                "unexpected number of points per segment {}".format(len(segment))
            )
    polygon = np.concatenate(polygon_segments)
    polygons.append(polygon)
for polygon in polygons:
    plt.plot(*polygon.T)

In [ ]:
np.concatenate(((0,), contours[:-1]))

In [ ]:
freetype.FT_CURVE_TAG_ON